<a href="https://colab.research.google.com/github/arjun-2131/cow-breed/blob/master/cow-breed_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Convolution2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation,Dropout,Flatten,Dense,BatchNormalization
from keras import backend as K
from keras.preprocessing import image
from keras import applications
from keras.callbacks import ModelCheckpoint,EarlyStopping

Using TensorFlow backend.


In [0]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
from keras.applications.xception import preprocess_input
import seaborn as sns
sns.set(color_codes=True)

In [0]:
batch_size=10
num_classes=5
epochs=100
img_width,img_height=256,256

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_data_dir="/content/drive/My Drive/cow/train"
validation_data_dir="/content/drive/My Drive/cow/valid"
test_data_dir="/content/drive/My Drive/cow/test"

In [0]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

In [7]:
  model=applications.xception.Xception(include_top=False,
                                      weights='imagenet',
                                      input_shape=(img_width,img_height,3))

W0620 04:50:50.307181 139667965446016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0620 04:50:50.354753 139667965446016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0620 04:50:50.361512 139667965446016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0620 04:50:50.400527 139667965446016 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0620 04:50:50.401318 1396679654

83689472/83683744 [==============================] - 1s 0us/step


In [8]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=BatchNormalization()(x)
x=Dense(1024,activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(0.5)(x)
y_pred=Dense(5,activation='softmax')(x)

W0620 04:51:00.888625 139667965446016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model_final=Model(inputs=model.input,outputs=y_pred)
model_final.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
for layers in model.layers[:126]:
  layers.trainable=False
for layer in model.layers[126:]:
  layer.trainable=True

In [0]:
train_datagen=ImageDataGenerator(
              rescale=1./255,
              shear_range=0.2,
              zoom_range=0.2,
              horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [12]:
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
validation_generator=test_datagen.flow_from_directory(validation_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
test_generator=test_datagen.flow_from_directory(test_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

Found 404 images belonging to 5 classes.
Found 41 images belonging to 5 classes.
Found 41 images belonging to 5 classes.


In [0]:
model_final.compile(loss='categorical_crossentropy',
                   optimizer='SGD',
                   metrics=['accuracy'])   

In [45]:
top_model_weight_path='/content/drive/My Drive/cow/Saved_Models/cow_Xception(V2).h5'
checkpoint=ModelCheckpoint(filepath=top_model_weight_path,
                          verbose=1,
                            save_best_only=True,
                          monitor='val_acc')
early=EarlyStopping(monitor='val_acc',min_delta=0,patience=5,verbose=1,mode='auto')

history=model_final.fit_generator(
              train_generator,
                steps_per_epoch=404//batch_size,
              epochs=epochs,
                validation_data=validation_generator,
                validation_steps=41//batch_size,
                callbacks=[checkpoint,early])

Epoch 1/100
40/40 [==============================] - 11s 263ms/step - loss: 0.2188 - acc: 0.9388 - val_loss: 0.9188 - val_acc: 0.7742

Epoch 00001: val_acc improved from -inf to 0.77419, saving model to /content/drive/My Drive/cow/Saved_Models/cow_Xception(V2).h5
Epoch 2/100
40/40 [==============================] - 11s 285ms/step - loss: 0.1500 - acc: 0.9550 - val_loss: 1.0536 - val_acc: 0.7419

Epoch 00002: val_acc did not improve from 0.77419
Epoch 3/100
40/40 [==============================] - 11s 264ms/step - loss: 0.2196 - acc: 0.9363 - val_loss: 0.9853 - val_acc: 0.7750

Epoch 00003: val_acc improved from 0.77419 to 0.77500, saving model to /content/drive/My Drive/cow/Saved_Models/cow_Xception(V2).h5
Epoch 4/100
40/40 [==============================] - 10s 261ms/step - loss: 0.1410 - acc: 0.9500 - val_loss: 0.8128 - val_acc: 0.7742

Epoch 00004: val_acc did not improve from 0.77500
Epoch 5/100
40/40 [==============================] - 11s 273ms/step - loss: 0.1929 - acc: 0.9438 - 

In [0]:
scores,acc=model_final.evaluate_generator(test_generator,steps=12)

In [44]:
img=image.load_img("/content/drive/My Drive/cow/imag2es.jpeg")
target_size=(256,256)
if img.size!=target_size:
  img=img.resize(target_size)
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)
preds=model_final.predict(x)
print("probabilities for sindhi,ongole,kangrej,khillari,gir:",preds[0])

probabilities for sindhi,ongole,kangrej,khillari,gir: [0.39039707 0.00114609 0.02048307 0.00127572 0.586698  ]


In [49]:
print (acc*100)

76.34408640605147


In [0]:
print(model_final.metrics_names)